**Authorization**

In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-YOUR PROJECT NAME')

**Calculate LST**

In [13]:
import ee
import folium
import google.generativeai as genai



# Get user input for city
print("Please enter the city name exactly as it appears in the GAUL database.")
print("Example format: 'Delhi', 'Mumbai', 'Chennai', 'Kolkata'")
city_name = input("Enter city name: ")

# Define the AOI as the boundary of the selected city
admin_boundaries = ee.FeatureCollection('FAO/GAUL/2015/level2')
city_boundary = admin_boundaries.filter(ee.Filter.eq('ADM2_NAME', city_name))
aoi = city_boundary.geometry()

# Verify if the city was found
if city_boundary.size().getInfo() == 0:
    raise ValueError(f"City '{city_name}' not found in database. Please check the spelling and try again.")

# Function to apply scale factors to Landsat 8 image
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# Function to mask clouds in Landsat 8 images
def mask_l8sr(image):
    cloud_shadow_bit_mask = (1 << 3)
    clouds_bit_mask = (1 << 5)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0).And(qa.bitwiseAnd(clouds_bit_mask).eq(0))
    return image.updateMask(mask)

# Function to calculate Land Surface Temperature (LST)
def calculate_lst(image):
    # Calculate NDVI
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

    # Calculate emissivity
    em = ndvi.expression(
        '(NDVIm < 0 ? 0.985 : (NDVIm > 0.7 ? 0.99 : 0.985 + 0.005 * NDVIm))', {
        'NDVIm': ndvi
    }).rename('EM')

    # Get thermal band
    thermal = image.select('ST_B10').rename('thermal')

    # Calculate LST
    lst = thermal.expression(
        '(TB / (1 + (0.00115 * (TB / 1.438)) * log(EM))) - 273.15', {
            'TB': thermal,
            'EM': em
    }).rename('LST')

    return lst

# Function to get image collection for a specific year
def get_image_collection(year):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'

    collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
        .filterDate(start_date, end_date)
        .filterBounds(aoi)
        .map(apply_scale_factors)
        .map(mask_l8sr))

    # Check if collection is empty
    if collection.size().getInfo() == 0:
        raise ValueError(f"No Landsat 8 images found for {city_name} in {year}")

    return collection

# Create a Folium map centered on the city
city_center = aoi.centroid().coordinates().getInfo()
Map = folium.Map(location=[city_center[1], city_center[0]], zoom_start=11)

# Add base layer
folium.TileLayer(
    tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
    attr='OpenStreetMap',
    name='OpenStreetMap',
    overlay=False,
    control=True
).add_to(Map)

# Process each year
years = ['2019', '2020', '2021', '2022', '2023']
lst_values = {}

# Visualization parameters
lst_vis = {
    'min': 25,
    'max': 50,
    'palette': [
        '#040274', '#040281', '#0502a3', '#0502b8', '#0502ce', '#0502e6',
        '#0602ff', '#235cb1', '#307ef3', '#269db1', '#30c8e2', '#32d3ef',
        '#3be285', '#3ff38f', '#86e26f', '#3ae237', '#b5e22e', '#d6e21f',
        '#fff705', '#ffd611', '#ffb613', '#ff8b13', '#ff6e08', '#ff500d',
        '#ff0000', '#de0101', '#c21301', '#a71001', '#911003'
    ],
    'opacity': 0.3
}

def add_ee_layer(image, vis_params, name):
    map_id_dict = ee.Image(image).getMapId(vis_params)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(Map)

# Process each year
for year in years:
    try:
        print(f"\nProcessing year: {year}")
        collection = get_image_collection(year)

        # Get median image for the year
        median_image = collection.median()

        # Calculate LST
        lst_image = calculate_lst(median_image).clip(aoi)

        # Calculate mean LST
        lst_mean = lst_image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=30,
            maxPixels=1e9
        ).get('LST').getInfo()

        lst_values[year] = lst_mean
        print(f"Mean LST for {year}: {lst_mean:.2f}°C")

        # Add layer to map
        add_ee_layer(lst_image, lst_vis, f'LST {year}')

    except Exception as e:
        print(f"Error processing year {year}: {str(e)}")
        lst_values[year] = None

# Add layer control
folium.LayerControl().add_to(Map)

# Generate analysis using Gemini if we have valid data
if any(lst_values.values()):
    # Calculate additional statistics
    valid_temps = [v for v in lst_values.values() if v is not None]
    if valid_temps:
        max_temp = max(valid_temps)
        min_temp = min(valid_temps)
        avg_temp = sum(valid_temps) / len(valid_temps)
        temp_range = max_temp - min_temp

        # Calculate year-over-year changes
        changes = []
        years_list = list(lst_values.keys())
        for i in range(1, len(years_list)):
            if lst_values[years_list[i]] is not None and lst_values[years_list[i-1]] is not None:
                change = lst_values[years_list[i]] - lst_values[years_list[i-1]]
                changes.append(change)

    # Create temperature data string
    temp_data = []
    for year in years:
        if lst_values[year] is not None:
            temp_data.append(f"- {year}: {lst_values[year]:.2f}°C")
        else:
            temp_data.append(f"- {year}: No data available")
    temp_data_str = "\n".join(temp_data)

    # Create changes string
    changes_str = ", ".join([f"{change:+.2f}°C" for change in changes])

# Display the map
Map

Please enter the city name exactly as it appears in the GAUL database.
Example format: 'Delhi', 'Mumbai', 'Chennai', 'Kolkata'
Enter city name: Bangalore Urban

Processing year: 2019
Mean LST for 2019: 35.83°C

Processing year: 2020
Mean LST for 2020: 33.19°C

Processing year: 2021
Mean LST for 2021: 34.11°C

Processing year: 2022
Mean LST for 2022: 34.30°C

Processing year: 2023
Mean LST for 2023: 34.80°C


**Albedo and Tree Cover**

In [11]:
import ee
import folium
import time
from google.colab import drive

# Authenticate and initialize Earth Engine


# Get user input for city
city_name = input("Enter city name: ")

# Define the AOI as the boundary of the selected city
admin_boundaries = ee.FeatureCollection('FAO/GAUL/2015/level2')
city_boundary = admin_boundaries.filter(ee.Filter.eq('ADM2_NAME', city_name))
aoi = city_boundary.geometry()

if city_boundary.size().getInfo() == 0:
    raise ValueError(f"City '{city_name}' not found in the database. Please check spelling and try again.")

# Function to apply scale factors to Landsat 8 image
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    return image.addBands(optical_bands, None, True)

# Function to mask clouds
def mask_l8sr(image):
    cloud_shadow_bit_mask = (1 << 3)
    clouds_bit_mask = (1 << 5)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0).And(qa.bitwiseAnd(clouds_bit_mask).eq(0))
    return image.updateMask(mask)

# Function to calculate NDVI and classify vegetation density
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    vegetation_classes = ndvi.expression(
        "(b('NDVI') < 0.2) ? 1 : (b('NDVI') < 0.5) ? 2 : 3"
    ).rename('VegetationDensity')
    return vegetation_classes

# Function to calculate albedo
def calculate_albedo(image):
    coefficients = {'SR_B2': 0.356, 'SR_B3': 0.130, 'SR_B4': 0.373, 'SR_B5': 0.085, 'SR_B6': 0.072, 'SR_B7': -0.0018}
    albedo = ee.Image(0)
    for band, coeff in coefficients.items():
        albedo = albedo.add(image.select(band).multiply(coeff))
    albedo = albedo.add(0.016).max(0).min(1).multiply(100).rename('albedo')
    return albedo

# Get image collection for a specific year with cloud cover filtering
def get_image_collection(year):
    start_date, end_date = f'{year}-01-01', f'{year}-12-31'
    collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                  .filterDate(start_date, end_date)
                  .filterBounds(aoi)
                  .filter(ee.Filter.lt('CLOUD_COVER', 20))  # Filter by cloud cover
                  .map(apply_scale_factors)
                  .map(mask_l8sr))
    if collection.size().getInfo() == 0:
        raise ValueError(f"No Landsat 8 images found for {city_name} in {year}")
    return collection

# Visualization parameters
vegetation_vis = {'min': 1, 'max': 3, 'palette': ['#d73027', '#fee08b', '#1a9850']}
albedo_vis = {'min': 0, 'max': 60, 'palette': ['#2C1A5A', '#4B2991', '#6B3894', '#8B4984', '#AB5C6C', '#CB7152', '#EB8A36', '#FBA21B', '#F8C91E', '#F2E627', '#FFFFFF']}

# Create map
city_center = aoi.centroid().coordinates().getInfo()
Map = folium.Map(location=[city_center[1], city_center[0]], zoom_start=11)

# Function to add EE layers to folium map
def add_ee_layer(image, vis_params, name):
    map_id_dict = ee.Image(image).getMapId(vis_params)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(Map)

# Extract statistics function
def extract_statistics(image, aoi):
    mean_value = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=30,
        maxPixels=1e10
    ).getInfo()
    return mean_value

# Temporal analysis for multiple years
def temporal_analysis(start_year, end_year):
    temporal_vegetation = []
    temporal_albedo = []

    for year in range(start_year, end_year + 1):
        print(f"Processing year: {year}")
        start_time = time.time()

        collection = get_image_collection(str(year))
        median_image = collection.median()
        vegetation_image = calculate_ndvi(median_image).clip(aoi)
        albedo_image = calculate_albedo(median_image).clip(aoi)

        vegetation_mean = extract_statistics(vegetation_image, aoi)
        albedo_mean = extract_statistics(albedo_image, aoi)

        temporal_vegetation.append(vegetation_mean['VegetationDensity'])
        temporal_albedo.append(albedo_mean['albedo'])

        print(f"Year {year} completed in {time.time() - start_time:.2f} seconds.")

    return temporal_vegetation, temporal_albedo

# Run temporal analysis from 2015 to 2020
try:
    temporal_vegetation, temporal_albedo = temporal_analysis(2015, 2020)
    print(f"Temporal Vegetation Density (2015-2020): {temporal_vegetation}")
    print(f"Temporal Albedo (2015-2020): {temporal_albedo}")
except Exception as e:
    print(f"Error during temporal analysis: {e}")

# Process the latest year (2023)
try:
    year = '2023'
    print(f"Processing year: {year}")
    collection = get_image_collection(year)
    median_image = collection.median()

    # Calculate vegetation density clusters and albedo
    vegetation_image = calculate_ndvi(median_image).clip(aoi)
    albedo_image = calculate_albedo(median_image).clip(aoi)

    # Add layers to map for the latest year
    add_ee_layer(vegetation_image, vegetation_vis, 'Vegetation Density (2023)')
    add_ee_layer(albedo_image, albedo_vis, 'Albedo (2023)')
    folium.LayerControl().add_to(Map)

    print(f"Year {year} completed.")
except Exception as e:
    print(f"Error processing year {year}: {e}")

# Add legend to map
legend_html = """
<div style="
    position: fixed;
    top: 50%; left: 20px;
    transform: translateY(-50%);
    width: 220px;
    background-color: white;
    z-index: 9999;
    font-size: 14px;
    border: 2px solid grey;
    padding: 10px;
    box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
">
    <b>Legend</b><br>
    <div style="display: flex; align-items: center;">
        <div style="width: 18px; height: 18px; background: #d73027; margin-right: 8px;"></div> Low Vegetation
    </div>
    <div style="display: flex; align-items: center;">
        <div style="width: 18px; height: 18px; background: #fee08b; margin-right: 8px;"></div> Medium Vegetation
    </div>
    <div style="display: flex; align-items: center;">
        <div style="width: 18px; height: 18px; background: #1a9850; margin-right: 8px;"></div> High Vegetation
    </div>
    <br>
    <b>Albedo Scale:</b><br>
    <div style="display: flex; align-items: center;">
        <div style="width: 18px; height: 18px; background: #2C1A5A; margin-right: 8px;"></div> Low Albedo
    </div>
    <div style="display: flex; align-items: center;">
        <div style="width: 18px; height: 18px; background: #F2E627; margin-right: 8px;"></div> High Albedo
    </div>
</div>
"""

Map.get_root().html.add_child(folium.Element(legend_html))

# Add a compass rose (direction scale) to the map
compass_rose_html = """
<div style="
    position: fixed;
    top: 50%; right: 50px;
    transform: translateY(-50%);
    width: 200px; height: 200px;
    z-index: 9999;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Brosen_windrose.svg/300px-Brosen_windrose.svg.png"
         style="width: 100%; height: 100%;"/>
</div>
"""

Map.get_root().html.add_child(folium.Element(compass_rose_html))

# Display map
Map

Enter city name: Bangalore Urban
Processing year: 2015
Year 2015 completed in 13.66 seconds.
Processing year: 2016
Year 2016 completed in 11.56 seconds.
Processing year: 2017
Year 2017 completed in 13.18 seconds.
Processing year: 2018
Year 2018 completed in 14.17 seconds.
Processing year: 2019
Year 2019 completed in 59.11 seconds.
Processing year: 2020
Year 2020 completed in 13.30 seconds.
Temporal Vegetation Density (2015-2020): [2.041670547135771, 2.021252900278324, 2.027967482946649, 2.088773884914322, 1.9949284118998392, 2.0850001436807832]
Temporal Albedo (2015-2020): [13.391842399922055, 13.355705141400797, 13.014227232948453, 12.478866963908315, 13.094189992803598, 12.72196211184222]
Processing year: 2023
Year 2023 completed.


In [14]:
#!/usr/bin/env python3
"""
Land Surface Temperature (LST) Analysis using Google Earth Engine and Landsat 8
================================================================================

This script performs multi-year Land Surface Temperature analysis for urban areas
using Landsat 8 satellite data. It generates interactive maps showing temperature
patterns and temporal trends.

Features:
- Multi-year LST calculation (2019-2023)
- Cloud masking and quality filtering
- Interactive Folium map visualization
- Statistical analysis and trend detection
- Integration with Google Generative AI for insights

Dependencies:
- earthengine-api
- folium
- google-generativeai

Author: Your Name
License: MIT
"""

import ee
import folium
import google.generativeai as genai
from typing import Dict, List, Optional, Tuple


class LSTAnalyzer:
    """Land Surface Temperature analyzer for urban climate studies."""

    def __init__(self):
        """Initialize the LST analyzer with Earth Engine authentication."""
        try:
            ee.Initialize()
        except Exception:
            print("Earth Engine authentication required. Please run ee.Authenticate() first.")
            raise

    def get_city_boundary(self, city_name: str) -> ee.Geometry:
        """
        Get city boundary from FAO GAUL database.

        Args:
            city_name (str): Name of the city as it appears in GAUL database

        Returns:
            ee.Geometry: City boundary geometry

        Raises:
            ValueError: If city is not found in database
        """
        admin_boundaries = ee.FeatureCollection('FAO/GAUL/2015/level2')
        city_boundary = admin_boundaries.filter(ee.Filter.eq('ADM2_NAME', city_name))

        if city_boundary.size().getInfo() == 0:
            raise ValueError(f"City '{city_name}' not found in database. Please check the spelling and try again.")

        return city_boundary.geometry()

    def apply_scale_factors(self, image: ee.Image) -> ee.Image:
        """
        Apply scale factors to Landsat 8 Collection 2 surface reflectance data.

        Args:
            image (ee.Image): Landsat 8 image

        Returns:
            ee.Image: Scaled image
        """
        optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
        thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
        return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

    def mask_clouds(self, image: ee.Image) -> ee.Image:
        """
        Mask clouds and cloud shadows in Landsat 8 images.

        Args:
            image (ee.Image): Landsat 8 image with QA_PIXEL band

        Returns:
            ee.Image: Cloud-masked image
        """
        cloud_shadow_bit_mask = (1 << 3)
        clouds_bit_mask = (1 << 5)
        qa = image.select('QA_PIXEL')
        mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0).And(
            qa.bitwiseAnd(clouds_bit_mask).eq(0)
        )
        return image.updateMask(mask)

    def calculate_lst(self, image: ee.Image) -> ee.Image:
        """
        Calculate Land Surface Temperature using single-channel algorithm.

        Args:
            image (ee.Image): Landsat 8 image with surface reflectance and thermal bands

        Returns:
            ee.Image: LST image in Celsius
        """
        # Calculate NDVI for emissivity estimation
        ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

        # Calculate emissivity based on NDVI
        emissivity = ndvi.expression(
            '(NDVIm < 0 ? 0.985 : (NDVIm > 0.7 ? 0.99 : 0.985 + 0.005 * NDVIm))', {
                'NDVIm': ndvi
            }
        ).rename('EM')

        # Get thermal band (converted to Kelvin)
        thermal = image.select('ST_B10').rename('thermal')

        # Calculate LST using Planck's law inversion
        lst = thermal.expression(
            '(TB / (1 + (0.00115 * (TB / 1.438)) * log(EM))) - 273.15', {
                'TB': thermal,
                'EM': emissivity
            }
        ).rename('LST')

        return lst

    def get_image_collection(self, year: str, aoi: ee.Geometry) -> ee.ImageCollection:
        """
        Get filtered Landsat 8 image collection for a specific year and area.

        Args:
            year (str): Year to filter (e.g., '2020')
            aoi (ee.Geometry): Area of interest

        Returns:
            ee.ImageCollection: Filtered image collection

        Raises:
            ValueError: If no images found for the specified criteria
        """
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'

        collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                     .filterDate(start_date, end_date)
                     .filterBounds(aoi)
                     .map(self.apply_scale_factors)
                     .map(self.mask_clouds))

        if collection.size().getInfo() == 0:
            raise ValueError(f"No Landsat 8 images found for the specified area in {year}")

        return collection

    def add_ee_layer(self, folium_map: folium.Map, image: ee.Image,
                     vis_params: Dict, name: str) -> None:
        """
        Add Earth Engine layer to Folium map.

        Args:
            folium_map (folium.Map): Folium map object
            image (ee.Image): Earth Engine image
            vis_params (Dict): Visualization parameters
            name (str): Layer name
        """
        map_id_dict = ee.Image(image).getMapId(vis_params)
        folium.TileLayer(
            tiles=map_id_dict['tile_fetcher'].url_format,
            attr='Google Earth Engine',
            name=name,
            overlay=True,
            control=True
        ).add_to(folium_map)

    def analyze_lst_trends(self, city_name: str, years: List[str] = None) -> Tuple[folium.Map, Dict]:
        """
        Perform comprehensive LST analysis for multiple years.

        Args:
            city_name (str): Name of the city to analyze
            years (List[str], optional): Years to analyze. Defaults to 2019-2023.

        Returns:
            Tuple[folium.Map, Dict]: Interactive map and LST statistics
        """
        if years is None:
            years = ['2019', '2020', '2021', '2022', '2023']

        # Get area of interest
        aoi = self.get_city_boundary(city_name)

        # Create base map
        city_center = aoi.centroid().coordinates().getInfo()
        folium_map = folium.Map(
            location=[city_center[1], city_center[0]],
            zoom_start=11
        )

        # Add base layer
        folium.TileLayer(
            tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
            attr='OpenStreetMap',
            name='OpenStreetMap',
            overlay=False,
            control=True
        ).add_to(folium_map)

        # LST visualization parameters
        lst_vis = {
            'min': 25,
            'max': 50,
            'palette': [
                '#040274', '#040281', '#0502a3', '#0502b8', '#0502ce', '#0502e6',
                '#0602ff', '#235cb1', '#307ef3', '#269db1', '#30c8e2', '#32d3ef',
                '#3be285', '#3ff38f', '#86e26f', '#3ae237', '#b5e22e', '#d6e21f',
                '#fff705', '#ffd611', '#ffb613', '#ff8b13', '#ff6e08', '#ff500d',
                '#ff0000', '#de0101', '#c21301', '#a71001', '#911003'
            ],
            'opacity': 0.3
        }

        # Process each year
        lst_values = {}
        for year in years:
            try:
                print(f"Processing year: {year}")
                collection = self.get_image_collection(year, aoi)

                # Get median composite
                median_image = collection.median()

                # Calculate LST
                lst_image = self.calculate_lst(median_image).clip(aoi)

                # Calculate mean LST for the area
                lst_mean = lst_image.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=aoi,
                    scale=30,
                    maxPixels=1e9
                ).get('LST').getInfo()

                lst_values[year] = lst_mean
                print(f"Mean LST for {year}: {lst_mean:.2f}°C")

                # Add layer to map
                self.add_ee_layer(folium_map, lst_image, lst_vis, f'LST {year}')

            except Exception as e:
                print(f"Error processing year {year}: {str(e)}")
                lst_values[year] = None

        # Add layer control
        folium.LayerControl().add_to(folium_map)

        # Calculate statistics
        stats = self._calculate_statistics(lst_values)

        return folium_map, {'lst_values': lst_values, 'statistics': stats}

    def _calculate_statistics(self, lst_values: Dict[str, Optional[float]]) -> Dict:
        """
        Calculate statistical measures from LST data.

        Args:
            lst_values (Dict[str, Optional[float]]): LST values by year

        Returns:
            Dict: Statistical measures
        """
        valid_temps = [v for v in lst_values.values() if v is not None]

        if not valid_temps:
            return {}

        stats = {
            'max_temp': max(valid_temps),
            'min_temp': min(valid_temps),
            'avg_temp': sum(valid_temps) / len(valid_temps),
            'temp_range': max(valid_temps) - min(valid_temps)
        }

        # Calculate year-over-year changes
        years_list = list(lst_values.keys())
        changes = []
        for i in range(1, len(years_list)):
            if (lst_values[years_list[i]] is not None and
                lst_values[years_list[i-1]] is not None):
                change = lst_values[years_list[i]] - lst_values[years_list[i-1]]
                changes.append(change)

        if changes:
            stats['year_over_year_changes'] = changes
            stats['avg_annual_change'] = sum(changes) / len(changes)

        return stats


def main():
    """Main function to run LST analysis."""
    print("Land Surface Temperature Analysis")
    print("=" * 40)
    print("Please enter the city name exactly as it appears in the GAUL database.")
    print("Example format: 'Delhi', 'Mumbai', 'Chennai', 'Kolkata'")

    city_name = input("Enter city name: ").strip()

    if not city_name:
        print("City name cannot be empty!")
        return

    try:
        # Initialize analyzer
        analyzer = LSTAnalyzer()

        # Perform analysis
        print(f"\nAnalyzing LST trends for {city_name}...")
        folium_map, results = analyzer.analyze_lst_trends(city_name)

        # Display results
        print("\n" + "=" * 50)
        print("ANALYSIS RESULTS")
        print("=" * 50)

        lst_values = results['lst_values']
        stats = results['statistics']

        # Print yearly values
        print("\nYearly Mean LST:")
        for year, temp in lst_values.items():
            if temp is not None:
                print(f"  {year}: {temp:.2f}°C")
            else:
                print(f"  {year}: No data available")

        # Print statistics
        if stats:
            print(f"\nStatistical Summary:")
            print(f"  Maximum Temperature: {stats['max_temp']:.2f}°C")
            print(f"  Minimum Temperature: {stats['min_temp']:.2f}°C")
            print(f"  Average Temperature: {stats['avg_temp']:.2f}°C")
            print(f"  Temperature Range: {stats['temp_range']:.2f}°C")

            if 'avg_annual_change' in stats:
                print(f"  Average Annual Change: {stats['avg_annual_change']:+.2f}°C/year")

        # Save map
        map_filename = f"lst_analysis_{city_name.lower().replace(' ', '_')}.html"
        folium_map.save(map_filename)
        print(f"\nInteractive map saved as: {map_filename}")

        # Display the map (in Jupyter environments)
        return folium_map

    except Exception as e:
        print(f"Error during analysis: {str(e)}")
        print("Please check your input and try again.")


if __name__ == "__main__":
    main()

Land Surface Temperature Analysis
Please enter the city name exactly as it appears in the GAUL database.
Example format: 'Delhi', 'Mumbai', 'Chennai', 'Kolkata'
Enter city name: Bangalore Urban
Earth Engine authentication required. Please run ee.Authenticate() first.
Error during analysis: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.
Please check your input and try again.
